In [1]:
from project_paths import *

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GroupShuffleSplit, GridSearchCV, KFold, ParameterGrid
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, fbeta_score, accuracy_score

In [2]:
data = pd.read_csv(data_csv_for_preprocessing)

print(data.shape)
data.head()

(39644, 51)


,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,day_of_week,topic,popular
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593,Monday,Entertainment,0
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711,Monday,Business,0
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500,Monday,Business,1
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200,Monday,Entertainment,0
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505,Monday,Tech,0


In [3]:
# Preprocessor
onehot_ftrs = load_list_from_pkl('onehot_ftrs.pkl')
minmax_ftrs = load_list_from_pkl('minmax_ftrs.pkl')
standard_ftrs = load_list_from_pkl('standard_ftrs.pkl')

preprocessor = ColumnTransformer(
    transformers=[('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'), onehot_ftrs), 
                 ('minmax', MinMaxScaler(), minmax_ftrs),
                 ('standard', StandardScaler(), standard_ftrs),])

#### First we split the data into other and test to maintain a separate Test set

In [4]:
TEST_SPLIT_RANDOM_STATE = 99

non_predictive_columns = ['url', 'timedelta']
target_column = 'popular'

y = data[target_column]
X = data[[x for x in data.columns if x not in non_predictive_columns and x != target_column]]

# Splitting the data into other vs holdout (90 vs 10) - BASIC TRAIN TEST SPLIT
X_other, X_holdout, y_other, y_holdout = train_test_split(X, y, train_size=0.9, random_state=TEST_SPLIT_RANDOM_STATE)

print(X_other.shape)
print(X_holdout.shape)

print(y_other.shape)
print(y_holdout.shape)

(35679, 48)
(3965, 48)
(35679,)
(3965,)


In [31]:
35679*0.1, 35679*0.1, 35679*0.9

(3567.9, 3567.9, 32111.100000000002)

In [26]:
save_list_to_pkl(X_other, 'X_other.pkl')
save_list_to_pkl(X_holdout, 'X_holdout.pkl')
save_list_to_pkl(y_other, 'y_other.pkl')
save_list_to_pkl(y_holdout, 'y_holdout.pkl')

#### Next, we build the ML Pipeline function which can accept the other dataset, a preprocessor, an ML Model and a parameter grid and perform gridsearch over the hyperparameters over several random states and return the best models (by validation score) for each random state

In [5]:
# EVAL_METRIC = "accuracy"
EVAL_METRIC = "f_beta"

In [6]:
def MLpipe_KFold_f_beta_15(X, y, preprocessor, ML_algo, param_grid):
    '''
    This function creates an other vs test set. 
    Then applies KFold with 9 folds on other.
    The f_beta(1.5) is maximized in cross-validation.
    Then run the fitted model on the test set. 
    The above steps are performed once for each of 10 random states. 
    '''
    
    # Repeat this 10 times for 10 different random states, and 
    # the function should return the 10 best models and the 10 test scores.
    test_scores = []
    best_models = []
    best_params = []
    cv_results = []
    BETA = 1.5
    random_states = [x*5 for x in list(range(10))]
    for RANDOM_STATE in random_states:

        # Inside the function, split the data to other and test 
        # (We already kept 10% aside earlier as a holdout set. From remaining 90%: 80% for train/val + 10% for test)
        X_other, X_test, y_other, y_test = train_test_split(X, y, train_size = 0.89, random_state=RANDOM_STATE)

        # Then preprocess the data 
        pipe = Pipeline(steps=[
                         ('preprocessor', preprocessor),
                         ('regressor', ML_algo)
                    ])

        # and then use KFold with 8 folds. (from the 80% for train/val)
        kfold_split = KFold(n_splits=8, shuffle=True, random_state=RANDOM_STATE)

        # Using an f_beta score with beta=1.5 as the evaluation metric
        if EVAL_METRIC == "accuracy":
            score_function = make_scorer(accuracy_score)
        else:
            score_function = make_scorer(fbeta_score, beta=BETA)

        # perform cross validation
        grid = GridSearchCV(pipe, param_grid=param_grid, scoring = score_function,
                            cv=kfold_split, return_train_score = True, n_jobs=-1, verbose=True)
#         grid = GridSearchCV(pipe, param_grid=param_grid, scoring = score_function,
#                             cv=kfold_split, return_train_score = True, verbose=True)

        grid.fit(X_other, y_other)

        parameters_of_best_fit = grid.best_params_
        best_fit_model = grid.best_estimator_
        best_params.append(parameters_of_best_fit)
        best_models.append(best_fit_model)
        
        # Cross validation results of best parameter set 
        # (we will get 1 for each random state)
        cv_result = pd.DataFrame(grid.cv_results_)
        cv_result.iloc[grid.best_index_].to_dict()
        cv_results.append(cv_result)
        
        test_score = fbeta_score(y_test, grid.predict(X_test), beta=BETA)
        test_scores.append(test_score)

    return best_models, test_scores, cv_results


In [7]:
# Dictionary to track each model's performance
MODEL_EVALUATIONS = {}

In [8]:
from sklearn.linear_model import LogisticRegression

param_grid = {
    'regressor__max_iter': [10000], 
}

LR_model = LogisticRegression()
best_models, test_scores, cv_results = MLpipe_KFold_f_beta_15(X_other, y_other, preprocessor, LR_model, param_grid)

# best_model, best_score, random_state_wise_stdev = find_best_model_on_test(X_holdout, y_holdout, best_models)

MODEL_EVALUATIONS['logistic_regression'] = {
    'best_models': best_models, 
    'test_scores': test_scores,
    'cv_results': cv_results,
}

save_list_to_pkl(MODEL_EVALUATIONS, MODEL_EVALUATIONS_PICKLE)

Fitting 8 folds for each of 1 candidates, totalling 8 fits
Fitting 8 folds for each of 1 candidates, totalling 8 fits
Fitting 8 folds for each of 1 candidates, totalling 8 fits
Fitting 8 folds for each of 1 candidates, totalling 8 fits
Fitting 8 folds for each of 1 candidates, totalling 8 fits
Fitting 8 folds for each of 1 candidates, totalling 8 fits
Fitting 8 folds for each of 1 candidates, totalling 8 fits
Fitting 8 folds for each of 1 candidates, totalling 8 fits
Fitting 8 folds for each of 1 candidates, totalling 8 fits
Fitting 8 folds for each of 1 candidates, totalling 8 fits


## This is remaining -> takes too long, run later

In [36]:
from sklearn.linear_model import LogisticRegression

param_grid = {
    'regressor__max_iter': [10000], 
    'regressor__penalty': ["l1"], 
    'regressor__solver': ["saga"], 
    'regressor__C': [1e-2, 1e-1, 1, 1e1, 1e2], 
}

LassoR_model = LogisticRegression()
# best_models, test_scores, cv_results = MLpipe_KFold_f_beta_15(X_other, y_other, preprocessor, LassoR_model, param_grid)

# # best_model, best_score, random_state_wise_stdev = find_best_model_on_test(X_holdout, y_holdout, best_models)

# MODEL_EVALUATIONS['lasso'] = {
#     'best_models': best_models, 
#     'test_scores': test_scores,
#     'cv_results': cv_results,
# }

save_list_to_pkl(MODEL_EVALUATIONS, MODEL_EVALUATIONS_PICKLE)

In [9]:
from sklearn.linear_model import LogisticRegression

param_grid = {
    'regressor__max_iter': [10000], 
    'regressor__penalty': ["l2"], 
    'regressor__C': [1e-2, 1e-1, 1, 1e1, 1e2], 
}

RidgeR_model = LogisticRegression()
best_models, test_scores, cv_results = MLpipe_KFold_f_beta_15(X_other, y_other, preprocessor, RidgeR_model, param_grid)

# best_model, best_score, random_state_wise_stdev = find_best_model_on_test(X_holdout, y_holdout, best_models)

MODEL_EVALUATIONS['ridge'] = {
    'best_models': best_models, 
    'test_scores': test_scores,
    'cv_results': cv_results,
}

save_list_to_pkl(MODEL_EVALUATIONS, MODEL_EVALUATIONS_PICKLE)

Fitting 8 folds for each of 5 candidates, totalling 40 fits
Fitting 8 folds for each of 5 candidates, totalling 40 fits
Fitting 8 folds for each of 5 candidates, totalling 40 fits
Fitting 8 folds for each of 5 candidates, totalling 40 fits
Fitting 8 folds for each of 5 candidates, totalling 40 fits
Fitting 8 folds for each of 5 candidates, totalling 40 fits
Fitting 8 folds for each of 5 candidates, totalling 40 fits
Fitting 8 folds for each of 5 candidates, totalling 40 fits
Fitting 8 folds for each of 5 candidates, totalling 40 fits
Fitting 8 folds for each of 5 candidates, totalling 40 fits


In [13]:
from sklearn.ensemble import RandomForestClassifier

max_features = [0.5, 0.75, None]
max_depth = [5, 25, 100]
param_grid = {'regressor__max_features': max_features, 
              'regressor__max_depth': max_depth}
RF = RandomForestClassifier()

best_models, test_scores, cv_results = MLpipe_KFold_f_beta_15(X_other, y_other, preprocessor, RF, param_grid)

# best_model, best_score, random_state_wise_stdev = find_best_model_on_test(X_holdout, y_holdout, best_models)

MODEL_EVALUATIONS['random_forest'] = {
    'best_models': best_models, 
    'test_scores': test_scores,
    'cv_results': cv_results,
}

save_list_to_pkl(MODEL_EVALUATIONS, MODEL_EVALUATIONS_PICKLE)

# model_evals_random_forest = load_preprocessed_data(path_data_interim + "random_forest_model_evals.pkl.npy")
# best_model_random_forest = load_preprocessed_data(path_data_interim + "random_forest_best_model.pkl")
# best_models_random_forest = load_preprocessed_data(path_data_interim + "random_forest_best_models.pkl")
# test_scores_random_forest = load_preprocessed_data(path_data_interim + "random_forest_test_scores.pkl")

Fitting 8 folds for each of 9 candidates, totalling 72 fits
Fitting 8 folds for each of 9 candidates, totalling 72 fits
Fitting 8 folds for each of 9 candidates, totalling 72 fits
Fitting 8 folds for each of 9 candidates, totalling 72 fits
Fitting 8 folds for each of 9 candidates, totalling 72 fits
Fitting 8 folds for each of 9 candidates, totalling 72 fits
Fitting 8 folds for each of 9 candidates, totalling 72 fits
Fitting 8 folds for each of 9 candidates, totalling 72 fits
Fitting 8 folds for each of 9 candidates, totalling 72 fits
Fitting 8 folds for each of 9 candidates, totalling 72 fits


In [46]:
# save_preprocessed_data(MODEL_EVALUATIONS['random_forest'], path_data_interim + "random_forest_model_evals.pkl")
# save_preprocessed_data(best_model, path_data_interim + "random_forest_best_model.pkl")
# save_preprocessed_data(best_models, path_data_interim + "random_forest_best_models.pkl")
# save_preprocessed_data(test_scores, path_data_interim + "random_forest_test_scores.pkl")

## This is remaining -> takes too long, run later

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier()
ns = [2, 5, 10, 30]
param_grid = {'regressor__n_neighbors': ns}

# best_models, test_scores, cv_results = MLpipe_KFold_f_beta_15(X_other, y_other, preprocessor, KNN, param_grid)

# # best_model, best_score, random_state_wise_stdev = find_best_model_on_test(X_holdout, y_holdout, best_models)

# MODEL_EVALUATIONS['KNN'] = {
#     'best_models': best_models, 
#     'test_scores': test_scores,
#     'cv_results': cv_results,
# }

save_list_to_pkl(MODEL_EVALUATIONS, MODEL_EVALUATIONS_PICKLE)

### XGBoost

In [19]:
import xgboost
from sklearn.model_selection import KFold

def xgb_model(X_other, y_other, X_test, y_test, RANDOM_STATE, verbose=1):

    # make into row vectors to avoid an obnoxious sklearn/xgb warning
#     y_other = np.reshape(np.array(y_other), (1, -1)).ravel()
#     y_test = np.reshape(np.array(y_test), (1, -1)).ravel()

    # Parameter grid to loop over
    param_grid = {"learning_rate": [0.05],
              "n_estimators": [50, 100, 500, 1000, 2000],
              "seed": [RANDOM_STATE],
              "missing": [np.nan], 
              "max_depth": [1,3,10,30,100,],
              # "max_depth": [1,],
              "colsample_bytree": [0.9],              
              "subsample": [0.66]}

    BETA = 1.5
    score_function = make_scorer(fbeta_score, beta=BETA)
    XGB = xgboost.XGBClassifier(use_label_encoder=False, n_jobs=7, eval_metric=score_function)
    pg = ParameterGrid(param_grid)

    scores = np.zeros(len(pg))

    for i in range(len(pg)):
        if verbose >= 5:
            print("Param set " + str(i + 1) + " / " + str(len(pg)))
        params = pg[i]
        # print("params", params)
        # DO THE K-FOLD SPLIT HERE
        kf_scores = np.zeros(8)
        kf = KFold(n_splits=8,shuffle=True, random_state=RANDOM_STATE)
        kf_idx = 0
        for train_index, val_index in kf.split(X_other,y_other):
            X_train = X_other.iloc[train_index]
            y_train = y_other.iloc[train_index]
            # y_train = np.take(y_other, train_index)
            X_CV = X_other.iloc[val_index]
            y_CV = y_other.iloc[val_index]
            # y_CV = np.take(y_other, val_index)

            XGB.set_params(**params)
            eval_set = [(X_CV, y_CV)]
            XGB.fit(X_train, y_train,
                    early_stopping_rounds=50, eval_set=eval_set, verbose=False)# with early stopping
            y_CV_pred = XGB.predict(X_CV, ntree_limit=XGB.best_ntree_limit)
            kf_scores[kf_idx] = fbeta_score(y_CV, y_CV_pred, beta=BETA)
            kf_idx += 1
        scores[i] = np.mean(kf_scores)
        print(scores[i])

    # print("Scores")
    # print(scores)
    best_params = np.array(pg)[scores == np.max(scores)]
    print('Val set max score and best parameters are:')
    print(np.max(scores))
    print(best_params)

    # test the model on the test set with best parameter set
    XGB.set_params(**best_params[0])
    XGB.fit(X_train, y_train,
            early_stopping_rounds=50,eval_set=eval_set, verbose=False)
    # y_CV_pred = XGB.predict(X_CV, ntree_limit=XGB.best_ntree_limit)
    y_test_pred = XGB.predict(X_test, ntree_limit=XGB.best_ntree_limit)
    best_model = XGB
    best_score = fbeta_score(y_test, y_test_pred, beta=BETA)

    return best_model, best_params[0], best_score

 - Take X_other, y_other
 - Split into other vs test
 - Do 8-fold CV on other to find best params
 - Run best params model on test

In [20]:
test_scores = []
best_models = []
best_params = []
random_states = [x*5 for x in list(range(10))]
scores_max_depths = []

for RANDOM_STATE in random_states:
    print("RANDOM_STATE: ", RANDOM_STATE)

    # Splitting the data
    X_other_2, X_test, y_other_2, y_test = train_test_split(X_other, y_other, train_size = 0.89, random_state=RANDOM_STATE)
    # X_other_2, y_other_2 will be sent to the xgb_model function for k-fold CV

    # preprocessing
    X_other_2_prep = preprocessor.fit_transform(X_other_2)
    X_test_prep = preprocessor.transform(X_test)

    feature_names = list(preprocessor.named_transformers_['onehot'].get_feature_names()) + \
                preprocessor.transformers[1][-1] + preprocessor.transformers[2][-1]

    df_other = pd.DataFrame(data=X_other_2_prep, columns = feature_names)
    df_test = pd.DataFrame(data=X_test_prep, columns = feature_names)

    best_model, best_params_found, best_test_score = \
            xgb_model(df_other, y_other_2, df_test, y_test, RANDOM_STATE, verbose=10)
    test_scores.append(best_test_score)
    best_models.append(best_model)
    best_params.append(best_params_found)
    print()

MODEL_EVALUATIONS['xgboost'] = {
    'best_models': best_models, 
    'test_scores': test_scores,
}
save_list_to_pkl(MODEL_EVALUATIONS, MODEL_EVALUATIONS_PICKLE)

RANDOM_STATE:  0
Param set 1 / 25
[14:26:29] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:26:29] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:26:29] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:26:30] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric i

In [23]:
MODEL_EVALUATIONS['xgboost']['test_scores']

[0.6454899129667508,
 0.6605222734254992,
 0.6695542605134667,
 0.66349080846017,
 0.6619819245929052,
 0.6486464840621496,
 0.6639321188724012,
 0.6470153319324462,
 0.6492474749490996,
 0.676366386688428]

In [71]:
save_preprocessed_data(test_scores, path_data_interim + "test_scores_xgb.pkl")
save_preprocessed_data(best_models, path_data_interim + "best_models_xgb.pkl")
save_preprocessed_data(best_params, path_data_interim + "best_params_xgb.pkl")

After tuning, each grid search’s best model parameters were extracted and used for comparison on eval metric on a holdout set over 10 random states (to find the mean score and std dev in scores across random states).

Now that we have established the best model, lets find the best parameters by trying:
 - over each of 10 random states
 - over each parameter configuration, check the score on the holdout set
 - pick the best one

In [7]:
def find_best_model_on_test(X_holdout, y_holdout, best_models):
    """
    Accepts best candidate configurations for a given ML model, 
    runs them across random states on holdout data 
    and returns results of best one 
    (Best is found as model which has highest mean fbeta score across random states. 
    For this model, we return the mean and stdev of fbeta scores across random states.)
    """
    BETA = 1.5
    random_states = [x*5 for x in list(range(10))]
    models_scores = np.zeros((len(random_states), len(best_models)))
    # Populate scores for each random state for each model
    for rs_ix, RANDOM_STATE in enumerate(random_states):
        for m_ix, m in enumerate(best_models):
            holdout_preds = m.predict(X_holdout)
            if EVAL_METRIC == "accuracy":
                models_scores[rs_ix, m_ix] = accuracy_score(y_holdout, holdout_preds)
            else:
                models_scores[rs_ix, m_ix] = fbeta_score(y_holdout, holdout_preds, beta=BETA)
    
    model_wise_scores = models_scores.mean(axis=0)
    best_model = best_models[np.argmax(model_wise_scores)]
    best_score = np.max(model_wise_scores)
    random_state_wise_stdev = np.mean(models_scores.std(axis=1))
    return best_model, best_score, random_state_wise_stdev